#### Для любой модели из курса(кроме KNN) и данных на которых она строилась: примнить метод главных компонент к данным, и сравнить работу модели на исходных данных с работой на данных где в качестве признаков используются две главные компоненты

Попробуем модель градиентного бустинга на деревьях решений. 

Следующие 5 ячеек - код с урока

In [1]:
from sklearn.tree import DecisionTreeRegressor

from sklearn import model_selection
import numpy as np

In [2]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [3]:
def gb_predict(X, trees_list, coef_list, eta):
    # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
    # поэтому все деревья из списка trees_list уже являются дополнительными и 
    #при предсказании прибавляются с шагом eta
    return np.array([sum(((1 if i==0 else eta)* coef * alg.predict([x])[0] 
                          for i, (alg, coef) in enumerate(zip(trees_list, coef_list)))) 
                     for x in X])


def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)


def bias(y, z):
    return (y - z)


def gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta):
    
    # Деревья будем записывать в список
    trees = []
    
    # Будем записывать ошибки на обучающей и тестовой выборке на каждой итерации в список
    train_errors = []
    test_errors = []
    
    for i in range(n_trees):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)

        # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
        # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
        if len(trees) == 0:
            # обучаем первое дерево на обучающей выборке
            tree.fit(X_train, y_train)
            
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, coefs, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, coefs, eta)))
        else:
            # Получим ответы на текущей композиции
            target = gb_predict(X_train, trees, coefs, eta)
            
            # алгоритмы начиная со второго обучаем на сдвиг
            tree.fit(X_train, bias(y_train, target))
            
            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, coefs, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, coefs, eta)))

        trees.append(tree)
        
    return trees, train_errors, test_errors

In [51]:
# Число деревьев в ансамбле
n_trees = 10

# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees

# Максимальная глубина деревьев
max_depth = 3

# Шаг
eta = 0.05

trees, train_errors, test_errors = gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta)

In [52]:
def evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta):
    train_prediction = gb_predict(X_train, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тренировочной выборке: {mean_squared_error(y_train, train_prediction)}')

    test_prediction = gb_predict(X_test, trees, coefs, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев глубиной {max_depth} \
    с шагом {eta} на тестовой выборке: {mean_squared_error(y_test, test_prediction)}')
evaluate_alg(X_train, X_test, y_train, y_test, trees, coefs, eta)

Ошибка алгоритма из 10 деревьев глубиной 3     с шагом 0.05 на тренировочной выборке: 2730.2939022043424
Ошибка алгоритма из 10 деревьев глубиной 3     с шагом 0.05 на тестовой выборке: 3536.635095765551


#### Метод главных компонент:

In [68]:
def pca(X_train, X_test, num=2): # задаем кол-во главных компонент
    X = X_train.astype(float)
    mean=np.apply_along_axis(np.mean, 0, X)
    std=np.apply_along_axis(np.std, 0, X)
    X_normed_train=(X-mean[np.newaxis, :])/std[np.newaxis, :]
    X_normed_test=(X_test-mean[np.newaxis, :])/std[np.newaxis, :]
    
    # Найдем собственные векторы и собственные значения 
    covariance_matrix = X_normed_train.T.dot(X_normed_train)
    eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    # Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
    W = eig_pairs[0][1].reshape(X.shape[1],1)
    for i in range(1, num):
        W = np.hstack((W, eig_pairs[i][1].reshape(X.shape[1],1)))
    print(f'Матрица весов W:\n', W)
    
    return X_normed_train.dot(W), X_normed_test.dot(W)
    

In [75]:
new_X_train, new_X_test = pca(X_train, X_test, 2)

Матрица весов W:
 [[-0.21733032 -0.19995957]
 [-0.18502792  0.36896673]
 [-0.30467229  0.10274987]
 [-0.2674335   0.0261983 ]
 [-0.35117579 -0.5387591 ]
 [-0.36202134 -0.39797817]
 [ 0.27567002 -0.57589296]
 [-0.42412513  0.15330275]
 [-0.37272103  0.06823572]
 [-0.32404535  0.06463192]]


In [76]:
trees, train_errors, test_errors = gb_fit(n_trees, max_depth, new_X_train, new_X_test, y_train, y_test, coefs, eta)
evaluate_alg(new_X_train, new_X_test, y_train, y_test, trees, coefs, eta)

Ошибка алгоритма из 10 деревьев глубиной 3     с шагом 0.05 на тренировочной выборке: 3381.9390300975506
Ошибка алгоритма из 10 деревьев глубиной 3     с шагом 0.05 на тестовой выборке: 4369.338581952493


Двух главных компонент не хватает, модель на исходных данных работает лучше, чем при использовании 2х главных компонент. Наилучшие результаты достигаются при использовании 4 главных компонент